In [ ]:
# Following tutorial here: https://prashanth08.medium.com/building-your-retrieval-augmented-generation-rag-for-custom-llms-d5f95ed5ed7a

In [3]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [ ]:
import openai
openai.api_key = os.environ.get("OPENAI_API_KEY")
from llama_index.core import SimpleDirectoryReader, GPTVectorStoreIndex

In [ ]:
openai.api_key = os.environ.get("OPENAI_API_KEY")

In [ ]:
reader = SimpleDirectoryReader("/Users/brianfrechette/Library/Mobile Documents/com~apple~CloudDocs/dev/digital_twin/llm_rag_intro")

In [ ]:
index = GPTVectorStoreIndex.from_documents(reader.load_data())

In [ ]:
query_engine = index.as_query_engine(similarity_top_k=3)

In [ ]:
query = "Who is Brian?"

In [ ]:
response = query_engine.query(query)
print(response)

## Pinecone as vector store, OpenAI as embedding model, Langchain as framework

### First, let's create Pinecone index

In [11]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

In [2]:
import time

In [5]:
# Client configuration details
pinecone_conn = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))
spec = ServerlessSpec(cloud='aws', region='us-east-1')

In [21]:
index_names = [index["name"] for index in pinecone_conn.list_indexes()]
index_names 

['digital-twin']

In [22]:
# Check and delete any pre-existing indexes with the same name
index_name = "digital-twin"
# if pinecone_conn.has_index(index_name):  --> Getting error: AttributeError: 'PineconeGRPC' object has no attribute 'has_index'
if index_name in index_names:
    pinecone_conn.delete_index(index_name)

# Creating a new index (so that langchain can connect and upsert docs)
pinecone_conn.create_index(
    index_name,
    dimension=1536, # dimensionality of text-embedding-ada-002
    metric='dotproduct',
    spec=spec
)

while not pinecone_conn.describe_index(index_name).status['ready']:
    time.sleep(1)

In [7]:
index = pinecone_conn.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 0}},
 'total_vector_count': 0}

### Next, let's connect the pinecone db to a langchain vector store to chunk and add docs

In [1]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_pinecone import PineconeVectorStore

In [24]:
# Instantiate DirectoryLoader object to load inputs docs to be upserted to pinecone vector store
loader = DirectoryLoader(
    "/Users/brianfrechette/Library/Mobile Documents/com~apple~CloudDocs/dev/digital_twin/llm_rag_intro",
    glob="*.docx",
    show_progress=True
)
docs = loader.load()
len(docs)

100%|██████████| 1/1 [00:00<00:00, 13.36it/s]


1

In [25]:
# Chunk documents
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs_chunked = text_splitter.split_documents(docs)
len(docs_chunked)

6

In [26]:
embeddings = OpenAIEmbeddings()

In [27]:
# Initialize pinecone vector store by adding docs to the index
vector_store_from_docs = PineconeVectorStore.from_documents(
    docs_chunked,
    index_name=index_name,
    embedding=embeddings
)

In [12]:
pinecone_conn = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

In [17]:
vector_store_from_docs = PineconeVectorStore(
    index_name="digital-twin",
    embedding=OpenAIEmbeddings()
)

In [32]:
# vector_store_from_docs.add_documents(
#     docs_chunked,
#     index_name=index_name,
#     embedding="text-embedding-ada-002"
# )

['7e7c1bdf-acfc-4fca-aff5-10b459db0a39',
 'ecd96f2a-3213-4310-8b32-3e41e747afb9',
 'c656c18a-aa6b-4458-956c-8687cebe1622',
 '5751a02c-a433-49e3-be5b-a56f2496d511',
 '68ea448c-dbdf-482b-a4ca-c0ec6216adf0',
 '1b75b83a-79d8-4504-a5ab-4c3523887789']

In [34]:
# vector_store_from_docs.delete(ids=["0fbbadc6-7e58-4b86-8c14-383759bcf059"])

In [18]:
# Test querying the vector db
query = "What experience does Brian have?"
vector_store_from_docs.similarity_search(query)

[Document(id='7e7c1bdf-acfc-4fca-aff5-10b459db0a39', metadata={'source': '/Users/brianfrechette/Library/Mobile Documents/com~apple~CloudDocs/dev/digital_twin/llm_rag_intro/B_Frechette_Resume.docx'}, page_content='Brian Frechette\n\nWest Hartford, CT 06119\n\nbrianfrechette1@yahoo.com | (603) 718-2765\n\nwww.brianfrechette.me | www.linkedin.com/in/brian-frechette | www.github.com/brianfrechette3\n\nEDUCATION\n\nUniversity of Pennsylvania – Master of Science in Artificial Intelligence                                                         Online / Philadelphia, PA\n\nRelevant coursework: Deep Learning, Natural Language Processing (NLP), GPU Programming, ML                            Jan 2025 – TBD\n\nUniversity of Pennsylvania – Master of Science in Computer Information Technology                                Online / Philadelphia, PA\n\nRelevant coursework: Software Design, Data Structures, Algorithms, Analytics, Data Science, ML, AI             Aug 2022 – May 2024\n\nCumulative GPA:

In [36]:
vector_store_from_docs.similarity_search("What is Brian's phone number?")

[Document(id='7e7c1bdf-acfc-4fca-aff5-10b459db0a39', metadata={'source': '/Users/brianfrechette/Library/Mobile Documents/com~apple~CloudDocs/dev/digital_twin/llm_rag_intro/B_Frechette_Resume.docx'}, page_content='Brian Frechette\n\nWest Hartford, CT 06119\n\nbrianfrechette1@yahoo.com | (603) 718-2765\n\nwww.brianfrechette.me | www.linkedin.com/in/brian-frechette | www.github.com/brianfrechette3\n\nEDUCATION\n\nUniversity of Pennsylvania – Master of Science in Artificial Intelligence                                                         Online / Philadelphia, PA\n\nRelevant coursework: Deep Learning, Natural Language Processing (NLP), GPU Programming, ML                            Jan 2025 – TBD\n\nUniversity of Pennsylvania – Master of Science in Computer Information Technology                                Online / Philadelphia, PA\n\nRelevant coursework: Software Design, Data Structures, Algorithms, Analytics, Data Science, ML, AI             Aug 2022 – May 2024\n\nCumulative GPA:

In [37]:
vector_store_from_docs.similarity_search("What technical skills does Brian have?")

[Document(id='7e7c1bdf-acfc-4fca-aff5-10b459db0a39', metadata={'source': '/Users/brianfrechette/Library/Mobile Documents/com~apple~CloudDocs/dev/digital_twin/llm_rag_intro/B_Frechette_Resume.docx'}, page_content='Brian Frechette\n\nWest Hartford, CT 06119\n\nbrianfrechette1@yahoo.com | (603) 718-2765\n\nwww.brianfrechette.me | www.linkedin.com/in/brian-frechette | www.github.com/brianfrechette3\n\nEDUCATION\n\nUniversity of Pennsylvania – Master of Science in Artificial Intelligence                                                         Online / Philadelphia, PA\n\nRelevant coursework: Deep Learning, Natural Language Processing (NLP), GPU Programming, ML                            Jan 2025 – TBD\n\nUniversity of Pennsylvania – Master of Science in Computer Information Technology                                Online / Philadelphia, PA\n\nRelevant coursework: Software Design, Data Structures, Algorithms, Analytics, Data Science, ML, AI             Aug 2022 – May 2024\n\nCumulative GPA:

### Vector store is loaded up with 6 chunked docs from my resume, let's try using langchains automated RAG features

In [19]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

In [20]:
llm = ChatOpenAI(
    openai_api_key=os.environ.get("OPENAI_API_KEY"),
    model_name="gpt-3.5-turbo",
    temperature=0.0
)

In [21]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store_from_docs.as_retriever()
)

In [22]:
qa.invoke("What is Brian's phone number?")

{'query': "What is Brian's phone number?",
 'result': "Brian's phone number is (603) 718-2765."}

In [23]:
qa.invoke("Can you tell me about Brian's most recent experience?")

{'query': "Can you tell me about Brian's most recent experience?",
 'result': "Brian's most recent experience is as a Senior Analytics Engineer at Raytheon Technologies (RTX) - Pratt & Whitney in East Hartford, CT. In this role, he serves as a technical lead in a cross-functional project to harmonize several enterprise tools into a central platform. The goal is to decrease tool fragmentation, increase tool scalability, and provide a single source of truth for data outputs. He has been in this position since May 2024."}

In [43]:
qa.invoke("Who does Brian hate?")

{'query': 'Who does Brian hate?',
 'result': "I don't have any information about who Brian Frechette hates."}

In [44]:
qa.invoke("What's the best fruit?")

{'query': "What's the best fruit?", 'result': "I don't know."}

In [46]:
qa.invoke("What was michael jordan's number?")

{'query': "What was michael jordan's number?",
 'result': "Michael Jordan's number was 23."}

In [24]:
qa.invoke("Who is Brian?")

{'query': 'Who is Brian?',
 'result': 'Brian Frechette is an individual with a background in engineering and analytics. He has pursued education in Artificial Intelligence and Computer Information Technology, with a strong focus on machine learning and data analysis. Brian has worked as an Analytics Engineer at Raytheon Technologies, where he has been involved in projects related to predictive modeling, data analysis, and process optimization.'}

In [25]:
qa.invoke("Why should I hire him?")

{'query': 'Why should I hire him?',
 'result': "Based on the provided information, you should consider hiring Brian Frechette because he has a strong background in analytics, machine learning, and software development. He has experience in developing predictive models, optimizing processes, and analyzing data to drive business decisions. Brian's proficiency in Python, SQL, and various data libraries showcases his technical skills. Additionally, his educational background in Artificial Intelligence, Computer Information Technology, and Engineering demonstrates a solid foundation in relevant fields. Overall, Brian's experience, skills, and education make him a valuable candidate for roles that require expertise in analytics, machine learning, and software development."}